In [1]:
import wrf
import netCDF4 as nc
import os
import glob
import numpy as np
import json

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

with open('C:/Users/dario/Documents/MATLABdott/WRF_IDEAL_SIMUL/configAMPSIT.json') as config_file:
    config = json.load(config_file)

path =config['input_pathname']
output_path = config['output_pathname']
sim= config['ncfile_format'] + '*'
files=glob.glob((path + sim))
files = [file.replace("\\", "/") for file in files]
regions=config['regions']


nsnapshots=config['totalsim']

#################################
######COORDINATES DEFINITION#####
#################################
num_points = len(regions)
vect_y = [0] * num_points
vect_x = [0] * num_points

for i in range(num_points):
    y_field = f'y{i+1}'
    x_field = f'x{i+1}'
    vect_y[i] = config[y_field] 
    vect_x[i] = config[x_field] 
#################################


#################################
######VARIABLES DEFINITION#######
#################################
variable_list = config['variables'] #any variable of interest goes here
variable_dim = config['is_3d'] #1: atmospheric variable (time,z,y,x), 0: surface variable(time,y,x)
#################################


#################################
######3D SECTION DEFINITION######
#################################
START_Z=0 #z start
END_Z=config['verticalmax'] #z end
NZ=END_Z-START_Z #number of z grid points
start_t=0 #time start
end_t=start_t+config['totalhours']

###################################

#########################################################################################
######LOAD OF NEEDED VARIABLES in ndarrays of size (simulation,variable,time,z,y,x)######
#########################################################################################

nv = len(variable_list) #number of variables
nr = len(regions)
hgt_cv=np.empty(nr) #terrain elevation, 0 if flat
zn_m=np.empty((nsnapshots,nr,NZ)) #middle point for vertically staggered variables (visualization purposes)
v=np.empty((nsnapshots,nr,nv,end_t,NZ)) #variable of interest   
m=1

sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1]))
for i in range(nsnapshots): #loop over all simulations
    nc_data = nc.Dataset(sorted_files[i])
    for k in range(0,nr):
        y_start = vect_y[k] - m
        y_end = vect_y[k] + m + 1
        hgt_cv[k] = np.squeeze(nc_data['HGT'][0, vect_y[k], vect_x[k]]) #took from files[0], terrain is assumed constant in time
        zn = np.squeeze(nc_data['PH'][0, START_Z:END_Z + 1, vect_y[k], vect_x[k]] + nc_data['PHB'][0, START_Z:END_Z + 1, vect_y[k], vect_x[k]]) / 9.81 #pressure-to-height conversion
        zn_m[i,k, :] = ((zn[1:] + zn[:-1]) / 2)
        for j, variable in enumerate(variable_list): #loop over all variables
            if variable_dim[j] == 1:
                #v[i, k, j, :, :] = np.squeeze(nc_data[variable][start_t:end_t, START_Z:END_Z, vect_y[k], vect_x[k]])
                v[i, k, j, :, :] = np.mean(np.squeeze(nc_data[variable][start_t:end_t, START_Z:END_Z, y_start:y_end, vect_x[k]]), axis=(2))
            elif variable_dim[j] == 0:
                #v[i, k, j, :, 0] = np.squeeze(nc_data[variable][start_t:end_t, vect_y[k], vect_x[k]]) #here all elements outside v[i, j, :, 0, :, :] are 0
                v[i, k, j, :, 0] = np.mean(np.squeeze(nc_data[variable][start_t:end_t, y_start:y_end, vect_x[k]]), axis=(1))
    nc_data.close()



In [2]:

var=np.empty((nsnapshots))
for k in range(nr):
    for j, variable in enumerate(variable_list):
        for z in range(END_Z):
            for t in range(end_t):
                output_filename = os.path.join(output_path, f"{variable}_{regions[k]}_lev{z+1}_{t+1}.txt")
                
                data_string = ','.join(map(str, v[:, k, j, t, z]))
                
                #print(data_string)
                #print(output_filename)
                
                with open(output_filename, 'w') as file:
                    file.write(data_string)
##########################################################################################
        
